# GARLIC demo

General-purpose Adaptive Richardson-Lucy Image Characterisation

# 1. General-purpose

## Import libraries and scripts

In [ ]:
%matplotlib ipympl
from matplotlib import pyplot as plt
from matplotlib import colors
from matplotlib.ticker import AutoMinorLocator

import ipywidgets as widgets
from IPython.display import display

import numpy as np
from time import time
from scipy import ndimage, special

import importlib
import scripts
importlib.reload(scripts)

Plotting:

In [ ]:
def new_figure(fig_name, figsize=(10, 5), nrows=1, ncols=1, sharex='col', sharey='row', gridspec_kw={'hspace': 0, 'wspace': 0}):
    plt.close(fig_name)
    fig = plt.figure(fig_name, figsize=figsize)
    axes = fig.subplots(nrows=nrows, ncols=ncols, squeeze=False,
                        sharex=sharex, sharey=sharey,
                        gridspec_kw=gridspec_kw
                       )
    fig.set_tight_layout(True)
    for ax in axes.flat:
        ax.xaxis.set_minor_locator(AutoMinorLocator())
        ax.yaxis.set_minor_locator(AutoMinorLocator())
        ax.tick_params(which='both', bottom=True, top=True, left=True, right=True)
        ax.tick_params(which='major', direction='inout', length=8, grid_alpha=.3)
        ax.tick_params(which='minor', direction='in', length=2, grid_alpha=.1)
        ax.grid(True, which='both')

    fig.suptitle(fig_name)
    
    return fig, axes

In [ ]:
default_cmap = plt.get_cmap("gist_earth").copy()
default_cmap.set_bad('gray')


def colour_map(ax, cblabel, data, cmap=default_cmap, norm=None, xlabel=None, x=None, ylabel=None, y=None):
    
    sigmas = np.linspace(-3, 3, 7)
    percentiles = 50 + 50 * special.erf(sigmas / np.sqrt(2))
    ticks = np.nanpercentile(data, percentiles)
    if norm is None:
        if ticks[-1] > 0:
            linthresh = np.median(data[data > 0])
            norm = colors.SymLogNorm(vmin=ticks[0], vmax=ticks[-1], linthresh=linthresh)
        else:
            norm = colors.Normalize(vmin=ticks[0], vmax=ticks[-1])

    if y is None:
        y = np.arange(data.shape[0])
    if x is None:
        x = np.arange(data.shape[1])

    im = ax.imshow(data,
                   extent=(x[0]-(x[1]-x[0])/2, x[-1]+(x[-1]-x[-2])/2, y[0]-(y[1]-y[0])/2, y[-1]+(y[-1]-y[-2])/2),
                   interpolation='nearest', origin='lower',
                   cmap=cmap,
                   norm=norm,
                  )
    #ax.set_aspect('auto')
    if xlabel is not None:
        ax.set_xlabel(xlabel)
    if ylabel is not None:
        ax.set_ylabel(ylabel)

    cb = fig.colorbar(im, ax=ax, orientation='vertical', shrink=.9)
    cb.ax.set_ylabel(cblabel)
    if ticks is not None:
        cb.ax.set_yticks(ticks=ticks, labels=[f'{value:.3g} ({percent:.1f}%)' for value, percent in zip(ticks, percentiles)])
    cb.ax.tick_params(labelsize='small')
    
    return im, cb, norm


## Read data

In [ ]:
importlib.reload(scripts.read_data)
object_name, data, true_spectrum = scripts.read_data.run(22, (0, 0, 1))
data_offset = np.nanmin(data)

## Parameters

In [ ]:
residual_accuracy = .01
max_iter = 100
kernel_truncation = 6

In [ ]:
#prob_threshold = .5
peak_threshold = 1.3
accretion_threshold = .5

# 2. Adaptive Richardson Lucy

Find noise, source, and background scales

In [ ]:
importlib.reload(scripts.diffuse_emission)
noise_scale = scripts.diffuse_emission.find_scale(data)
source_scale = scripts.diffuse_emission.find_scale(ndimage.gaussian_filter(data, noise_scale, truncate=kernel_truncation))
diffuse_scale = scripts.diffuse_emission.find_scale(ndimage.gaussian_filter(data, source_scale, truncate=kernel_truncation))
print(f'Scales: noise = {noise_scale:.2f}, sources = {source_scale:.2f}, diffuse emission = {diffuse_scale:.2f}')

In [ ]:
baseline = np.nanmin(data)
print(f'baseline={baseline:4g}')

In [ ]:
importlib.reload(scripts.multiscale_RL)
smoothing_radii = np.array([noise_scale, source_scale, diffuse_scale]) / np.sqrt(8*np.log(2)) # FWHM -> Gaussian sigma
#smoothing_radii = np.array([noise_scale, source_scale])
n_radii = smoothing_radii.size

mRL = scripts.multiscale_RL.run(data - baseline, smoothing_radii)
RL = np.sum(mRL, axis=0)
m_model = np.empty_like(mRL)
for i, radius in enumerate(smoothing_radii):
    m_model[i] = ndimage.gaussian_filter(mRL[i], radius, truncate=kernel_truncation)
model = baseline + np.sum(m_model, axis=0)

# 3. Image characterisation

## Noise

In [ ]:
residual = data - model
noise = np.sqrt(ndimage.gaussian_filter(residual**2, smoothing_radii[0], truncate=kernel_truncation)) #- ndimage.gaussian_filter(residual, diffuse_scale))
mean = np.nanmean(noise)
noise = np.where(np.isfinite(noise), noise, mean)
print(f'noise: {mean:.3g} +- {np.std(noise):.3g} [{np.min(noise):.3g} - {np.max(noise):.3g}]')

In [ ]:
fig_name = 'noise'
plt.close(fig_name)
fig = plt.figure(fig_name, figsize=(14, 3))
axes = fig.subplots(nrows=1, ncols=4, squeeze=False, sharex=True, sharey=True)
fig.suptitle(fig_name)
fig.set_tight_layout(True)

ax = axes[0, 0]
norm = colour_map(ax, 'data', data)[2]

ax = axes[0, 1]
colour_map(ax, 'model', model, norm=norm)

ax = axes[0, 2]
#colour_map(ax, 'residual', residual, norm=norm)
colour_map(ax, 'residual / noise', residual / noise, cmap='turbo_r', norm=colors.Normalize(vmin=-3, vmax=3))

ax = axes[0, 3]
#colour_map(ax, 'noise', noise, norm=norm)
colour_map(ax, 'noise', noise, cmap='inferno')

plt.show()

## Background

In [ ]:
pixel_type = np.argmax(m_model, axis=0)

In [ ]:
background = np.fmin(m_model[-1], mRL[-1])
background = ndimage.gaussian_filter(background, smoothing_radii[-1])
background = np.nanmedian([m_model[-1], mRL[-1], background], axis=0)
background = ndimage.gaussian_filter(background, smoothing_radii[-1])
background = baseline + smoothing_radii.size*background

In [ ]:
signal = model - background
SN = signal / noise
median_SN = np.nanmedian(SN)

In [ ]:
fig_name = 'signal_and_background'
plt.close(fig_name)
fig = plt.figure(fig_name, figsize=(8, 5))
axes = fig.subplots(nrows=2, ncols=2, squeeze=False, sharex=True, sharey=True)
fig.suptitle(fig_name)
fig.set_tight_layout(True)


ax = axes[0, 0]
norm = colour_map(ax, 'data', data)[2]

ax = axes[0, 1]
colour_map(ax, 'signal', signal)

ax = axes[1, 0]
colour_map(ax, 'background', background, norm=norm)

ax = axes[1, 1]
#colour_map(ax, 'noise', noise, norm=norm)
#colour_map(ax, 'noise', noise, cmap='inferno')
colour_map(ax, 'S/N', SN, cmap='gnuplot')
ax.contour(SN, levels=[peak_threshold], colors=['k'])
#ax.contour(SN, levels=[accretion_threshold, peak_threshold], colors=['k', 'k'], linestyles=['-', '--'])

plt.show()

In [ ]:
raise -1

# --- TESTS

## Signal probability

In [ ]:
mSN = (baseline + 3*m_model - model[np.newaxis, :]) / noise[np.newaxis, :]
mp_signal = 1 - np.exp(-.5*mSN**2)
#p_signal = 1 - np.sqrt(ndimage.gaussian_filter(np.nanmean((1 - mp_signal)**2, axis=0), source_scale, truncate=kernel_truncation))
p_signal = ndimage.gaussian_filter(np.nanmean(mp_signal, axis=0), smoothing_radii[0], truncate=kernel_truncation)
#p_signal = np.nanmean(mp_signal, axis=0)

In [ ]:
fig_name = 'multi-scale'
plt.close(fig_name)
fig = plt.figure(fig_name, figsize=(12, 8))
axes = fig.subplots(nrows=smoothing_radii.size+1, ncols=4, squeeze=False, sharex=True, sharey=True)
fig.suptitle(fig_name)
fig.set_tight_layout(True)


def plot_multi(col, thing, cmap=None, norm=None):
    lbl = ['compact', 'diffuse', 'background']
    for i in range(smoothing_radii.size):
        ax = axes[i, col]
        colour_map(ax, lbl[i], thing[i], cmap, norm)
    colour_map(axes[i+1, col], 'mean', np.nanmean(thing, axis=0), cmap, norm)

plot_multi(0, baseline + 3*mRL)
plot_multi(1, baseline + 3*m_model)
plot_multi(2, mSN, cmap='turbo_r', norm=colors.Normalize(vmin=-3, vmax=3))
plot_multi(3, mp_signal, cmap='turbo_r', norm=colors.Normalize(vmin=0, vmax=1))

#ax = axes[0, 0]
#ax.set_xlim(350, 400)
#ax.set_ylim(150, 200)

plt.show()

## Background subtraction

In [ ]:
background = baseline + 3*m_model[2]
converged = False
while not converged:
    old_bg = background
    background = ndimage.gaussian_filter(np.fmin(background, model), smoothing_radii[-1])
    change = (background - old_bg) / noise
    a, b = np.min(change), np.max(change)
    print(a, b)
    if a > -1:
        converged = True

final_model = p_signal * model + (1 - p_signal) * background = signal + background

In [ ]:
signal = p_signal * (model - background)

In [ ]:
signal = model - background

In [ ]:
np.nansum(data), np.nansum(model), np.nansum(background + signal)

In [ ]:
residual = data - (signal + background)
noise = np.sqrt(ndimage.gaussian_filter(residual**2, smoothing_radii[0], truncate=kernel_truncation)) #- ndimage.gaussian_filter(residual, background_scale))
mean = np.nanmean(noise)
noise = np.where(np.isfinite(noise), noise, mean)

In [ ]:
fig_name = 'signal_and_background'
plt.close(fig_name)
fig = plt.figure(fig_name, figsize=(12, 6))
axes = fig.subplots(nrows=2, ncols=3, squeeze=False, sharex=True, sharey=True)
fig.suptitle(fig_name)
fig.set_tight_layout(True)


ax = axes[0, 0]
norm = colour_map(ax, 'data', data)[2]

ax = axes[0, 1]
colour_map(ax, 'background', background, norm=norm)

ax = axes[0, 2]
#colour_map(ax, 'noise', noise, norm=norm)
colour_map(ax, 'noise', noise, cmap='inferno')


ax = axes[1, 0]
colour_map(ax, 'signal', signal)
ax.contour((p_signal > prob_threshold) & (signal > peak_threshold*noise), levels=[True], colors=['k'])

ax = axes[1, 1]
colour_map(ax, 'signal probability', p_signal, cmap='seismic_r', norm=colors.Normalize(vmin=0, vmax=1))

ax = axes[1, 2]
#colour_map(ax, 'signal', model*p_signal + background*(1 - p_signal))
colour_map(ax, 'S/N', p_signal*(model-background)/noise, cmap='seismic_r', norm=colors.Normalize(vmin=-2*peak_threshold, vmax=2*peak_threshold))


plt.show()

In [ ]:
np.nansum(data), np.sum(model), np.std(noise), np.mean(noise)

In [ ]:
np.nansum(data-baseline), np.nansum(model-baseline),  np.sum(RL)

# 3. Source finding

In [ ]:
hist, bins = np.histogram((SN_map).ravel(), bins=np.arange(SN_min, SN_max, .1), density=True)
x_bins = (bins[1:] + bins[:-1]) / 2
index_max = np.argmax(hist)
SN_mode = x_bins[index_max]
SN_threshold = 2*SN_mode - SN_min
'''
'''
,

fig, axes = new_figure('residual')


ax = axes[0, 0]
sc = ax.scatter(residual, baseline + background_estimate, s=1, alpha=.1, c=noise, cmap='jet')
cb = fig.colorbar(sc, ax=ax, orientation='vertical', shrink=.9)

plt.show()

In [ ]:
fig, axes = new_figure('signal-to-noise')#, nrows=2)


ax = axes[0, 0]
ax.plot(x_bins, hist, 'k-+')
ax.axvline(SN_mode, c='k', ls='--', label=f'mode={SN_mode:.2f}')
ax.axvline(SN_threshold, c='k', ls='-.', label=f'mode threshold={SN_threshold:.2f}')
ax.axhline(hist[index_max]/2, c='k', ls=':')

p16, p50 = np.nanpercentile(SN_map, [16, 50])
ax.axvline(p16, c='k', ls=':')
ax.axvline(p50, c='c', ls=':', label=f'median={p50:.2f}')
ax.axvline(2*p50-SN_min, c='k', ls=':', label=f'median threshold={2*p50-SN_min:.2f}')
ax.axvline(SN_mean, c='r', ls='--', label=f'mean={mu0:.2f} +- {std0:.2f}')
#ax.axvline(mu0_threshold, c='r', ls='-.', label=f'mu0_threshold={mu0_threshold:.2f}')

ax.legend()

'''
ax = axes[1, 0]
ax.scatter(residual/noise, background_estimate, s=1, alpha=.1)
ax.set_xlim(SN_min, SN_max)
'''

plt.show()

## Hierarchical Overdensity Tree

In [ ]:
importlib.reload(scripts.sort_data)
#argsorted_data, n_valid = scripts.sort_data.run(RL.ravel())
argsorted_data, n_valid = scripts.sort_data.run(mRL.ravel())

In [ ]:
importlib.reload(scripts.HOT)
sorted_strides = np.hstack([np.sort(mRL.strides)//mRL.itemsize, mRL.size]) # DIRTY HACK when testig particles at the boundary
t0 = time()
HOT_labels, HOT_catalog = scripts.HOT.run(mRL, argsorted_data, sorted_strides)
#n_sources = np.unique(HOT_labels).size
n_sources = np.max(HOT_labels)
print(f'     {time()-t0:.3g} seconds')

In [ ]:
HOT_parent = HOT_catalog[0]
'''
HOT_area = HOT_catalog[1]
HOT_test_stat = HOT_catalog[2]
HOT_bg = HOT_catalog[3]
#max_test_stat = catalog[3]
'''
,

## Individual sources

In [ ]:
def get_source_model(mRL, HOT_labels, lbl):
    RL = np.zeros_like(data)
    indices = np.where(HOT_labels[0] == lbl)
    RL[indices] = mRL[0][indices]
    source_model = ndimage.gaussian_filter(RL, smoothing_radii[0])
    
    RL = np.zeros_like(data)
    indices = np.where(HOT_labels[1] == lbl)
    RL[indices] = mRL[1][indices]
    source_model += ndimage.gaussian_filter(RL, smoothing_radii[1])
    
    return source_model

In [ ]:
def compute_moment(m):
    RL_moment = mRL.copy()
    for i, radius in enumerate(smoothing_radii):
        RL_moment[i] *= ndimage.gaussian_filter(residual**m / (compact_emission + diffuse_emission), radius)
    moment = np.zeros(n_sources+1)
    np.add.at(moment, HOT_labels, RL_moment)
    return moment

source_area = compute_moment(0)
source_residual = compute_moment(1)
mean_residual = source_residual / source_area
#rms_residual = compute_moment(2) / source_area

In [ ]:
source_SN = mean_residual * np.sqrt(source_area) / noise
sorted_by_SN = np.argsort(source_SN)
n_fluke = np.count_nonzero(np.cumsum(source_residual[sorted_by_SN]) < 0)
SN_threshold = source_SN[sorted_by_SN[n_fluke]]

In [ ]:
true_source = (source_SN > SN_threshold) & (source_area > compact_scale)
n_true = np.count_nonzero(true_source)
print(f'{n_true} true sourcces above S/N={SN_threshold:.1f} and area={compact_scale:.1f}')

In [ ]:
fig, axes = new_figure('source_flux', nrows=1, figsize=(12, 8))

ax = axes[0, 0]
ax.axhline(SN_threshold, color='k', ls=':', label=f'{SN_threshold:.2f} x {noise:.3g} = {SN_threshold*noise:.3g}')
ax.axvline(compact_scale, color='k', ls='--', label=f'compact scale = {compact_scale:.3g}')
ax.axvline(diffuse_scale, color='k', ls=':', label=f'diffuse scale = {diffuse_scale:.3g}')

ax.scatter(source_area[true_source], source_SN[true_source], c='b', s=.1, alpha=1)
ax.scatter(source_area[~true_source], source_SN[~true_source], c='r', s=.1, alpha=.5)
#for i in range(n_sources+1):
#    ax.text(source_residual[i], mean_residual[i], i, fontsize='x-small', clip_on=True)
ax.legend()
plt.show()

# 3. Plots

Normalisation and color maps:

In [ ]:
n_sources = np.unique(HOT_labels).size
latin_cube = np.vstack([(np.argsort(np.random.random(n_sources))+1)/n_sources, (np.argsort(np.random.random(n_sources))+1)/n_sources, (np.argsort(np.random.random(n_sources))+1)/n_sources, np.ones(n_sources)]).T
latin_cube[0, :] = [0., 0., 0., 1.]  # background object must be black :^)
label_cmap = colors.ListedColormap(latin_cube)
label_norm = colors.Normalize(vmin=-.5, vmax=n_sources+.5)
print(f'{n_sources} unique sourcces')

In [ ]:
mRL_cmap = default_cmap
mRL_norm = colors.LogNorm(vmin=np.percentile(mRL[mRL>0], 10), vmax=np.percentile(mRL[mRL>0], 99))

In [ ]:
fig_name = 'GARLIC_test'
plt.close(fig_name)
fig = plt.figure(fig_name, figsize=(15, 6))
axes = fig.subplots(nrows=2, ncols=4, squeeze=False, sharex=True, sharey=True)
fig.suptitle(fig_name)
fig.set_tight_layout(True)

combined_labels = HOT_labels[0].copy()
flukes = np.where(~true_source[combined_labels])
combined_labels[flukes] = HOT_labels[1][flukes]
flukes = np.where(~true_source[combined_labels])
combined_labels[flukes] = 0

ax = axes[0, 0]
colour_map(ax, 'compact', HOT_labels[0], cmap=label_cmap, norm=label_norm)
ax = axes[0, 1]
colour_map(ax, 'diffuse', HOT_labels[1], cmap=label_cmap, norm=label_norm)
ax = axes[0, 2]
colour_map(ax, 'combined', combined_labels, cmap=label_cmap, norm=label_norm)
ax = axes[0, 3]
colour_map(ax, 'data', data)

metric = source_SN
ax = axes[1, 0]
colour_map(ax, 'compact', metric[HOT_labels[0]])
ax = axes[1, 1]
colour_map(ax, 'diffuse', metric[HOT_labels[1]])
ax = axes[1, 2]
colour_map(ax, 'combined', metric[combined_labels])
ax = axes[1, 3]
colour_map(ax, 'residual', residual)

plt.show()

In [ ]:
np.count_nonzero(true_source)

In [ ]:
def get_mRL_inpaint(mRL, labels, target):

    mRL_target = np.zeros_like(mRL)
    indices = np.where(labels == target)
    mRL_target[indices] = mRL[indices]

    radius = np.sqrt(mRL.shape[0])
    inpaint_map = np.where(mRL_target > 0, 1., 0.)  # object mask
    inpaint_map = ndimage.gaussian_filter(inpaint_map, radius)  # interpolation weight
    inpaint_map = np.clip(inpaint_map, np.min(inpaint_map[inpaint_map > 0]), np.inf)  # to prevent division by zero
    inpaint_map = ndimage.gaussian_filter(mRL_target, radius) / inpaint_map

    return mRL_target, np.fmin(inpaint_map, mRL)


def get_individual_mRL(mRL, labels, parent, target):

    mRL_target, inpaint_map = get_mRL_inpaint(mRL, labels, target)
    
    # compute contribution to descendants:
    progenitor = parent[labels]
    if parent[target] == target:
        indices = np.where(labels == target)
        progenitor[indices] = 0
    n_found = 1
    while n_found > 0:
        indices = np.where(progenitor == target)
        n_found = len(indices[0])
        #print(f'{n_found} values painted')
        mRL_target[indices] = inpaint_map[indices]
        if parent[target] == target:
            progenitor[indices] = 0
        progenitor = parent[progenitor]

    # remove contribution from ancestors:
    '''
    RL[target] = mRL[target] - bg[lbl]
    '''

    return mRL_target


class Explore_lbl_1D(object):
    
    def __init__(self, fig_name, data, boosted_data, estimate, label, parent):
        """Interactive display"""
        
        plt.close(fig_name)
        self.fig = plt.figure(fig_name, figsize=(12, 7))
        self.axes = self.fig.subplots(nrows=4, ncols=2, squeeze=False, sharex='col', gridspec_kw={'width_ratios': [1, .02], 'hspace': 0})
        self.fig.suptitle(fig_name)
        self.fig.set_tight_layout(True)

        self.original = data
        self.boosted_data = boosted_data
        self.data_offset = np.nanmin(data)
        #self.RL = RL
        #self.SSF = SSF
        self.total_estimate = estimate
        self.label = label
        self.parent = parent

        self.ax_parent = self.axes[0, 0]
        self.ax_parent_cb = self.axes[0, 1]
        self.ax_lbl = self.axes[1, 0]
        self.ax_lbl_cb = self.axes[1, 1]

        self.ax_im = self.axes[2, 0]
        self.ax_cb = self.axes[2, 1]

        self.ax0 = self.axes[3, 0]
        self.ax0.plot(data, 'k-', alpha=.2)
        self.ax0.set_xlim(0, data.size)
        self.axes[3, 1].axis('off')

        self.widget = widgets.interactive(self.plot_lbl, lbl=widgets.BoundedIntText(value=1, min=1, max=n_sources, continuous_update=False))
        display(self.widget)


    def plot_lbl(self, lbl):
        xlim = self.ax0.get_xlim()
        ylim = self.ax0.get_ylim()

        ax = self.ax_parent
        im = ax.imshow(self.parent[self.label],
                       interpolation='nearest', origin='lower',
                       cmap=label_cmap, norm=colors.Normalize(vmin=-.5, vmax=n_sources+.5),
                      )
        ax.set_aspect('auto')
        #ax.set_ylim(-1, n_radii+1)
        ticks = np.array(ax.get_yticks(), dtype=int)#.clip(0, n_radii-1)
        ax.set_yticks(ticks)
        ax.set_yticklabels([f'{radius:.3g}' for radius in smoothing_radii[ticks]])
        cb = plt.colorbar(im, cax=self.ax_parent_cb, orientation='vertical', shrink=.9)
        cb.ax.tick_params(labelsize='small')
        cb.ax.set_ylabel('parent ID')
        
        
        ax = self.ax_lbl
        im = ax.imshow(self.label,
                       interpolation='nearest', origin='lower',
                       cmap=label_cmap, norm=colors.Normalize(vmin=-.5, vmax=n_sources+.5),
                      )
        ax.set_aspect('auto')
        #ax.set_ylim(-1, n_radii+1)
        ticks = np.array(ax.get_yticks(), dtype=int)#.clip(0, n_radii-1)
        ax.set_yticks(ticks)
        ax.set_yticklabels([f'{radius:.3g}' for radius in smoothing_radii[ticks]])
        cb = plt.colorbar(im, cax=self.ax_lbl_cb, orientation='vertical', shrink=.9)
        cb.ax.tick_params(labelsize='small')
        cb.ax.set_ylabel('source ID')

        
        RL = get_individual_mRL(mRL, HOT_labels, HOT_parent, lbl)
        self.ax_im.clear()
        im = self.ax_im.imshow(RL,
                               interpolation='nearest', origin='lower',
                               #cmap='gist_earth', norm=colors.LogNorm(vmin=np.min(RL[RL > 1e3*epsilon])))
                               #cmap=default_cmap, norm=colors.LogNorm(vmin=1/n_radii),
                               cmap=default_cmap, norm=colors.LogNorm(vmin=np.percentile(mRL[mRL>0], 10), vmax=np.percentile(mRL[mRL>0], 99)),
                               #cmap=default_cmap, norm=colors.LogNorm(vmin=np.percentile(RL[RL>0], 1), vmax=np.percentile(RL[RL>0], 99)),
                              )
        self.ax_im.set_aspect('auto')
        cb = plt.colorbar(im, cax=self.ax_cb, orientation='vertical', shrink=.9)
        cb.ax.tick_params(labelsize='small')
        self.fig.canvas.draw_idle()
        

        ax = self.ax0
        ax.clear()

        #RL = RL[np.newaxis, :] * self.SSF[:, np.newaxis]
        estimate = np.empty_like(RL)
        for i, radius in enumerate(smoothing_radii):
            #self.ax0.plot(original_pixel, RL[i], 'k-', alpha=.1)
            estimate[i] = ndimage.gaussian_filter(RL[i], radius)
        estimate = np.sum(estimate, axis=0)
        #print(np.min(estimate))
        
        fraction = estimate/self.total_estimate
        #estimate_data_offset = np.sum(estimate * ((self.boosted_data+self.data_offset)*fraction - estimate)) / np.sum(estimate)
        final_estimate = estimate #+ estimate_data_offset*np.sqrt(fraction/np.max(fraction))
        #print(estimate_data_offset, np.max(fraction))
        #estimate = self.boosted_data*fraction + self.data_offset*np.mean(fraction)
        #print(f'area: {np.sum(fraction):.2f} ({area[lbl]}),',
        #      f'flux: {np.nansum(final_estimate):.2f} ({test_stat[lbl]:.2f}),',
        mu = np.nansum(final_estimate) / np.count_nonzero(final_estimate > 0)
        print(f'area: {np.sum(fraction):.2f} ({np.count_nonzero(final_estimate[final_estimate <= mu] > 0)} + {np.count_nonzero(final_estimate[final_estimate > mu] > 0)} = {np.count_nonzero(final_estimate > 0)}),',
              f'flux: {np.nansum(final_estimate):.2f} ({np.nansum(final_estimate[final_estimate <= mu]):.2f}, {np.nansum(final_estimate[final_estimate > mu]):.2f}),',
              f'min-max:{np.min(self.boosted_data*fraction):.2f}-{np.max(self.boosted_data*fraction):.2f},'
              f'mean:{mu:.3f}',
              f'rms:{np.sum((fraction*self.boosted_data)**2)/np.sum(self.boosted_data*fraction):.3f}')
        print()

        #ax.plot(original_pixel, (self.boosted_data+self.data_offset)*fraction, 'r-.', alpha=.5)

        ax.plot(self.original, 'k-', alpha=.3)
        ax.plot(self.total_estimate + baseline, 'b-', alpha=.25)
        #ax.axhline(data_offset, c='r', ls=':', alpha=.5)
        ax.plot(baseline, 'y-', alpha=.5)
        ax.plot(baseline + residual_above_bg, 'y:', alpha=.5)
        ax.fill_between(np.arange(data.size), baseline, baseline+2*residual_above_bg, color='y', alpha=.1)

        ax.plot(np.sum(RL, axis=0) + baseline, 'r-', alpha=.25)
        ax.plot(estimate + baseline, 'g-', alpha=.5)


        ax.xaxis.set_minor_locator(AutoMinorLocator())
        ax.yaxis.set_minor_locator(AutoMinorLocator())
        ax.tick_params(which='both', bottom=True, top=True, left=True, right=True)
        ax.tick_params(which='major', direction='inout', length=8, grid_alpha=.3)
        ax.tick_params(which='minor', direction='in', length=2, grid_alpha=.1)
        ax.grid(True, which='both')
        #ax.set_yscale('log')
        ax.set_ylim(.9*self.data_offset, 3e5)
        ax.set_xlim(xlim)
        ax.set_ylim(ylim)
        #self.ax0.set_xlim(5000, 5300)


if len(data.shape) == 1:
    estimate = np.empty_like(mRL)
    for i, radius in enumerate(smoothing_radii):
        estimate[i] = ndimage.gaussian_filter(mRL[i], radius)
    estimate = np.sum(estimate, axis=0)
    rms_residual = np.std(boosted_sources - estimate)

    #for i, radius in enumerate(smoothing_radii):
    #     mRL[i] *= ndimage.gaussian_filter((boosted_data+epsilon) / (estimate+epsilon), radius)
    #RL = np.nansum(mRL, axis=0)
    #SSF_amplitude = np.nanmedian(mRL/RL[np.newaxis, :], axis=1)
    #print(rms_residual)
    x = Explore_lbl_1D(object_name, data, boosted_sources, estimate, HOT_labels, HOT_parent)

In [ ]:
residual_above_bg, residual_above_bg*data.size

In [ ]:
np.nansum(data-baseline), np.sqrt(np.nanstd(data-baseline-estimate)), np.nanmean(data-baseline-estimate), np.nanmedian(data-baseline-estimate)

In [ ]:
np.nanmedian(data-baseline), np.nanmean(data-baseline), np.nanstd(data-baseline-estimate)

In [ ]:
np.nanmedian(estimate), np.nanmean(estimate)

# --- OLD STUFF ---

In [ ]:
'''
fig_name = 'bg_test'
plt.close(fig_name)
fig = plt.figure(fig_name, figsize=(12,4))
axes = fig.subplots(nrows=1, ncols=1, squeeze=False)
fig.suptitle(fig_name)
fig.set_tight_layout(True)


ax = axes[0, 0]
ax.xaxis.set_minor_locator(AutoMinorLocator())
ax.yaxis.set_minor_locator(AutoMinorLocator())
ax.tick_params(which='both', bottom=True, top=True, left=True, right=True)
ax.tick_params(which='major', direction='inout', length=8, grid_alpha=.3)
ax.tick_params(which='minor', direction='in', length=2, grid_alpha=.1)
ax.grid(True, which='both')

ax.plot(true_spectrum, 'g-', alpha=.3)
ax.plot(data, 'k-', alpha=.2)
#ax.axhline(data_offset, c='r', ls=':', alpha=.5)
ax.plot(baseline, 'y-', alpha=.5)
ax.plot(baseline + residual_above_bg, 'y:', alpha=.5)
ax.fill_between(np.arange(data.size), baseline, baseline+2*residual_above_bg, color='y', alpha=.1)


ax.set_xlabel('pixel')
#ax.set_xlim(5900, 6300)
ax.set_ylim(np.min(baseline-residual_above_bg), np.max(baseline+4*residual_above_bg)*1.1)

plt.show()
'''
,

In [ ]:
importlib.reload(scripts.multiscale_RL)

compact_scale, diffuse_scale, baseline, residual_above_bg = scripts.diffuse_emission.run(data, max_iter=14)
smoothing_radii = np.array([1, compact_scale, diffuse_scale])
n_radii = smoothing_radii.size

n_iter = 0
mean_residual = 1
mean_below = 0
diffuse_emission = baseline
while n_iter < 10 and mean_residual > mean_below:
    n_iter += 1
    boosted_sources, mRL = scripts.multiscale_RL.run((data - diffuse_emission).clip(min=0), smoothing_radii)
    compact_emission = ndimage.gaussian_filter(mRL[0], 1) + ndimage.gaussian_filter(mRL[1], compact_scale)
    diffuse_residual = ndimage.gaussian_filter(mRL[2], diffuse_scale)
    total = np.nansum(data - baseline)
    mean_residual = np.nanmean(diffuse_residual)
    mean_below = np.nanmean((baseline - data)[data < diffuse_emission])
    print(np.nansum(compact_emission)/total, mean_residual, mean_below)
    diffuse_emission += diffuse_residual


In [ ]:
importlib.reload(scripts.diffuse_emission)
compact_scale, diffuse_scale, baseline, residual_above_bg = scripts.diffuse_emission.run(data, max_iter=14)

In [ ]:
#smoothing_radii = np.array([1, np.sqrt(compact_scale), compact_scale])
smoothing_radii = np.array([1, compact_scale, diffuse_scale])
n_radii = smoothing_radii.size
importlib.reload(scripts.multiscale_RL)
boosted_sources, mRL = scripts.multiscale_RL.run((data-baseline).clip(min=0), smoothing_radii)

In [ ]:
baseline += ndimage.gaussian_filter(mRL[2], diffuse_scale)
boosted_sources, mRL = scripts.multiscale_RL.run((data-baseline).clip(min=0), smoothing_radii)

In [ ]:
compact_emission = ndimage.gaussian_filter(mRL[0], 1) + ndimage.gaussian_filter(mRL[1], compact_scale)
diffuse_residual = ndimage.gaussian_filter(mRL[2], diffuse_scale)
estimate = baseline + np.sum(mRL, axis=0)
#index_minima = np.where(estimate < baseline + diffuse_residual + compact_emission)
#index_minima = scripts.diffuse_emission.find_minima(compact_emission)
index_minima = scripts.diffuse_emission.find_minima(estimate)

minima = estimate[index_minima] - baseline[index_minima]
p16, p50 = np.nanpercentile(minima, [16, 50])
mu0 = p50
var0 = (p50 - p16)**2
weight = np.exp(-.5 * (minima - mu0)**2 / var0)
total_weight = np.nansum(weight)
mu1 = np.nansum(weight * minima) / total_weight
var1 = np.nansum(weight * (minima- mu1)**2) / total_weight
var = 1 / (1/var1 - 1/var0)
mu = var * (mu1/var1 - mu0/var0)
diffuse_offset = mu
print(diffuse_offset, np.sqrt(var))

In [ ]:
diffuse_weight = (diffuse_residual / np.nansum(diffuse_residual)) / (compact_emission / np.nansum(compact_emission))
y = diffuse_weight * (data - baseline)
mu = np.nansum(y) / np.sum(diffuse_weight[np.isfinite(y)])
mu = 1

In [ ]:
diffuse_residual_norm = mu / np.nanmean(diffuse_residual)
diffuse_emission = baseline + diffuse_residual_norm*diffuse_residual

In [ ]:
diffuse_residual_norm = mu / np.nanmean(mRL[2][index_minima])
diffuse_emission = baseline + diffuse_residual_norm*diffuse_residual
